In [1]:
import pandas as pd
import numpy as np

# Importing the dataset from CSV file (Change the file name, dummy)
df = pd.read_csv('22_TCAP.csv')

# Concatenating TestGrade and ContentAreaCode fields
df['Grade_Subject'] = df['TestGrade'].astype(str) + '_' + df['ContentAreaCode']

# Calculating overall percentage of students who are proficient for each Grade_Subject and SchoolName
df['IsProficient'] = df['ProficiencyClassification'].isin(['Met Expectations', 'Exceeded Expectations'])
pivot_table = pd.pivot_table(df, values='IsProficient', index=['SchoolName', 'Grade_Subject'], aggfunc=np.mean)
pivot_table = pivot_table.reset_index()
pivot_table['Percentage Proficient'] = pivot_table['IsProficient'] * 100

# Writing pivot table data to separate CSV file
pivot_table.to_csv('22_TCAP_data.csv', index=False)


In [4]:
print(df['Grade_Subject'])

0       4_ENG
1       4_MAT
2       4_SCI
3       4_ENG
4       4_MAT
        ...  
7878    5_SCI
7879    7_ENG
7880    7_MAT
7881    7_SCI
7882    7_SOC
Name: Grade_Subject, Length: 7883, dtype: object


In [13]:
import pandas as pd
import numpy as np

# Importing the dataset from CSV file (Change the file name, dummy)
df = pd.read_csv('22_TCAP.csv')

# Concatenating TestGrade and ContentAreaCode fields
df['Grade_Subject'] = df['TestGrade'].astype(str) + '_' + df['ContentAreaCode']

# Determining which students are proficient
df['IsProficient'] = df['ProficiencyClassification'].isin(['Met Expectations', 'Exceeded Expectations'])

# Calculating percentage of proficient students by SchoolName and Grade_Subject
pivot_table = pd.pivot_table(df, values='IsProficient', index=['SchoolName', 'Grade_Subject'], aggfunc=np.mean)
pivot_table = pivot_table.reset_index()
pivot_table['Percentage Proficient'] = pivot_table['IsProficient'] * 100

# Filtering data to include only grades 3-5 and 6-8
df_filtered = df[(df['TestGrade'].isin(['03', '04', '05', '06', '07', '08']))]

# Grouping data by SchoolName, ContentAreaCode, and Literacy/Numeracy
df_filtered['Literacy/Numeracy'] = np.where(df_filtered['ContentAreaCode'].str.contains('ELA'), 'Literacy', 'Numeracy')
grouped = df_filtered.groupby(['SchoolName', 'Literacy/Numeracy'])

# Calculating percentage of proficient students by SchoolName and Literacy/Numeracy
grouped_table = pd.DataFrame({'IsProficient': grouped['IsProficient'].mean()})
grouped_table = grouped_table.reset_index()
grouped_table['Percentage Proficient'] = grouped_table['IsProficient'] * 100

# Renaming Literacy/Numeracy to Grade_Subject for consistency
grouped_table = grouped_table.rename(columns={'Literacy/Numeracy': 'Grade_Subject'})

# Concatenating proficiency tables for all grades and literacy/numeracy
all_tables = pd.concat([pivot_table, grouped_table])

# Creating new columns for overall Literacy and Numeracy proficiency for grades 3-5 and 6-8
overall_lit_3_5 = all_tables[(all_tables['Grade_Subject'].isin(['3_ENG', '4_ENG', '5_ENG'])) & (all_tables['Grade_Subject'].str.contains('Literacy'))]['Percentage Proficient'].mean()
overall_lit_6_8 = all_tables[(all_tables['Grade_Subject'].isin(['6_ENG', '7_ENG', '8_ENG'])) & (all_tables['Grade_Subject'].str.contains('Literacy'))]['Percentage Proficient'].mean()
overall_mat_3_5 = all_tables[(all_tables['Grade_Subject'].isin(['3_MAT', '4_MAT', '5_MAT'])) & (all_tables['Grade_Subject'].str.contains('Numeracy'))]['Percentage Proficient'].mean()
overall_mat_6_8 = all_tables[(all_tables['Grade_Subject'].isin(['6_MAT', '7_MAT', '8_MAT'])) & (all_tables['Grade_Subject'].str.contains('Numeracy'))]['Percentage Proficient'].mean()

# Creating new rows for overall Literacy and Numeracy proficiency
overall_lit_3_5_row = {'SchoolName': 'DISTRICT', 'Grade_Subject': '3-5_ENG', 'Percentage Proficient': overall_lit_3_5}
overall_lit_6_8_row = {'SchoolName': 'DISTRICT', 'Grade_Subject': '6-8_ENG', 'Percentage Proficient': overall_lit_6_8}
overall_mat_3_5_row = {'SchoolName': 'DISTRICT', 'Grade_Subject': '3-5_MAT', 'Percentage Proficient': overall_mat_3_5}
overall_mat_6_8_row = {'SchoolName': 'DISTRICT', 'Grade_Subject': '6-8_MAT', 'Percentage Proficient': overall_mat_6_8}

# Concatenating new rows to the proficiency table
new_rows = [pd.DataFrame([overall_lit_3_5_row]), pd.DataFrame([overall_lit_6_8_row]), pd.DataFrame([overall_mat_3_5_row]), pd.DataFrame([overall_mat_6_8_row])]
all_tables = pd.concat([all_tables] + new_rows, ignore_index=True)

# Writing updated proficiency table to CSV file
all_tables.to_csv('a.csv', index=False)




In [17]:
# Creating new DataFrame with the new rows
new_df = pd.DataFrame([overall_lit_3_5_row, overall_lit_6_8_row, overall_mat_3_5_row, overall_mat_6_8_row])

# Reading in the original data and concatenating with the new DataFrame
df = pd.read_csv('22_TCAP.csv')
df['Grade_Subject'] = df['TestGrade'].astype(str) + '_' + df['ContentAreaCode']
df['IsProficient'] = df['ProficiencyClassification'].isin(['Met Expectations', 'Exceeded Expectations'])
pivot_table = pd.pivot_table(df, values='IsProficient', index=['SchoolName', 'Grade_Subject'], aggfunc=np.mean)
pivot_table = pivot_table.reset_index()
all_tables = pd.concat([pivot_table, new_df], ignore_index=True)

# Writing updated DataFrame to CSV file
all_tables.to_csv('b.csv', index=False)
